In [1]:
import sys, os
sys.path.insert(0, '/data1/home/rocks/discrete_elasticity_v2/')
sys.path.insert(0, '/data1/home/rocks/discrete_elasticity_v2/python_src/')

import numpy as np
import scipy as sp
import scipy.optimize as opt
import numpy.random as rand
import numpy.linalg as la
import gzip
import cPickle as pickle
import matplotlib.pyplot as plt; plt.rcdefaults()
import seaborn as sns
%matplotlib inline

import mech_network_solver as mns
import network
import tuning_algs as talgs
import network_plot as nplot
import nonlin_solver as nls

sns.set(color_codes=True)
sns.set_context('poster', font_scale=1.75)
sns.set_palette("hls", 8)
# sns.set_palette("muted", 8)
sns.set_style('ticks', {'xtick.direction': 'in','ytick.direction': 'in', 'axes.linewidth': 2.0})

Loading Mechanical Network Deformation Solver Module


In [20]:
DIM = 2

eta = 0.01
NTS = 2
Lp = -1.0
NN = 64
irec = 1

nw_label = "network_periodic_jammed/network_N{:05d}_Lp{:.2f}/network_irec{:04d}".format(NN, Lp, irec)

with gzip.open("/data1/home/rocks/data/{:}.pklz".format(nw_label), 'rb') as pkl_file:

    nw_data = pickle.load(pkl_file)
    net = nw_data['network']   
    
    print net.DIM * net.NN
    print net.NE
    
rand.seed(13)

(success, inodesi, inodesj, onodesi, onodesj) = talgs.chooseFeasibleEdges(net, eta, NTS)
print success
print inodesi, inodesj, onodesi, onodesj

128
173
True
[[31]] [[44]] [[22, 13]] [[60, 54]]


In [21]:
NF = 1

isvec = [[] for t in range(NF)]
for t in range(NF):
    for (i, j) in zip(inodesi[t], inodesj[t]):
        posi = net.node_pos[DIM*i:DIM*i+DIM]
        posj = net.node_pos[DIM*j:DIM*j+DIM]
        bvec = posj - posi
        bvec -= np.round(bvec/net.L)*net.L
        isvec[t].extend(bvec) 

istrain = [[] for t in range(NF)]
istrain[0].append(1.0)


osvec = [[] for t in range(NF)]
ostrain = [[] for t in range(NF)]
for t in range(NF):
    for (i, j) in zip(onodesi[t], onodesj[t]):
        posi = net.node_pos[DIM*i:DIM*i+DIM]
        posj = net.node_pos[DIM*j:DIM*j+DIM]
        bvec = posj - posi
        bvec -= np.round(bvec/net.L)*net.L
        osvec[t].extend(bvec) 

        r = rand.randint(2)
        ostrain[t].append((2*r-1) * eta)

pert = []
meas = []
for t in range(NF):
    pert.append(talgs.Perturb())
    pert[t].setInputStrain(len(inodesi[t]), inodesi[t], inodesj[t], istrain[t], isvec[t])

    meas.append(talgs.Measure())
    meas[t].setOutputStrain(len(onodesi[t]), onodesi[t], onodesj[t], osvec[t])

obj_func = mns.CyIneqRatioChangeObjFunc(len(np.concatenate(ostrain)), net.NE, 
                                        np.zeros(len(np.concatenate(ostrain)), float), np.concatenate(ostrain))    


K_init = np.ones(net.NE, float) / net.eq_length

tuner = talgs.TuneContLin(net, pert, meas, obj_func, K_init)

cypert = []
for p in pert:
    cypert.append(p.getCyPert())

cymeas = []
for m in meas:
    cymeas.append(m.getCyMeas())

solver = mns.CyLinSolver(net.getCyNetwork(), len(cypert), cypert, cymeas)
solver.setIntStrengths(K_init)

(meas, grad) = solver.solveMeasGrad()

M = grad[0]

In [22]:
def func(dK):
    
    meas = M.dot(dK)
        
    return obj_func.objFunc(meas)
    
def grad(dK):
    
    meas = M.dot(dK)
    
    meas_grad = M
        
    return obj_func.objFuncGrad(meas, meas_grad)
    
tol = 10*np.finfo(float).eps
res = opt.minimize(func, np.zeros(net.NE, float), method='L-BFGS-B', callback=None, 
                   bounds=zip(-K_init+1e-6, np.zeros(net.NE, float)),
                       jac=grad, options={'ftol': tol, 'gtol': tol,'maxcor': 10,
                                                      'maxiter':10000000, 'maxfun': 10000000, 'maxls': 10000000})

print res.success, res.fun
dK = res.x

print dK

True 0.0
[ -1.86970126e-02   0.00000000e+00  -1.05773047e-01   0.00000000e+00
  -1.82848775e-02   0.00000000e+00  -4.35843671e-02  -4.25755324e-03
   0.00000000e+00  -4.21106870e-02   0.00000000e+00   0.00000000e+00
  -1.09797608e-02   0.00000000e+00   0.00000000e+00   0.00000000e+00
  -2.36864397e-01  -3.00804051e-03   0.00000000e+00   0.00000000e+00
  -7.11656221e-02  -2.53805905e-03  -2.00280830e-02   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00  -1.70342212e-01
   0.00000000e+00  -2.64105264e-01   0.00000000e+00  -2.07602356e-02
   0.00000000e+00  -2.29485420e-03  -5.83894571e-03   0.00000000e+00
  -2.65043104e-02  -2.08049267e-01   0.00000000e+00  -1.40332656e-03
  -4.12546521e-02  -5.62707472e-02  -1.01612713e-01  -5.70074833e-02
  -1.89295739e-02   0.00000000e+00  -1.64742640e-02   0.00000000e+00
  -1.09751148e-03   0.00000000e+00  -1.34151919e-01   0.00000000e+00
   0.00000000e+00  -4.89989805e-03   0.00000000e+00  -2.96279444e-01
  -1.25546432e-01   0.000

In [23]:
print tuner.func(K_init+dK)

0.0330015000086
